In [2]:
import pynwb
import numpy as np

nwb_path = r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"

with pynwb.NWBHDF5IO(nwb_path, "r") as io:
    nwb = io.read()
    
    # Raw ephys - this is the broadband data you can feed to KS4
    ephys = nwb.acquisition["extracellular array recording"]
    print(f"Shape: {ephys.data.shape}")       # (65033216, 56) = samples x channels
    print(f"Rate: {ephys.rate}")               # ~30kHz
    print(f"Starting time: {ephys.starting_time}")  # offset into video timebase
    
    # Already spike-sorted units
    print(f"\nUnits: {len(nwb.units)}")
    print(nwb.units.colnames)
    
    # To extract raw data for KS4 (careful - large!):
    raw = ephys.data[:]  # loads full array into RAM
    np.save("raw_ephys.npy", raw)

Shape: (65033216, 56)
Rate: 29999.989041455377
Starting time: 399.0283126054678

Units: 14
('inhibitory', 'depth', 'layer', 'neuron_channel', 'spike_times', 'obs_intervals')


In [2]:
from dandi.dandiapi import DandiAPIClient
from pathlib import Path

client = DandiAPIClient()
dandiset = client.get_dandiset("000231", "0.220904.1554")

for asset in dandiset.get_assets():
    if "KF134" in asset.path and "20180211T154918" in asset.path and "mkv" in asset.path:
        print(f"{asset.size/1e9:.2f} GB  {asset.path}")
        asset.download(Path(r"data\whisker_video_ephys.mkv"))
        break

6.60 GB  sub-KF134/sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv
